# Mini Social - Setup and Queries

This notebook connects to PostgreSQL, MongoDB, and Neo4j, then walks you through creating a tiny social media data model and running a few queries.

In [ ]:
import os
import pandas as pd
import time

# Connection info from environment
PGHOST = os.getenv("PGHOST", "localhost")
PGPORT = int(os.getenv("PGPORT", "5432"))
PGUSER = os.getenv("PGUSER", "app")
PGPASSWORD = os.getenv("PGPASSWORD", "app_pw")
PGDATABASE = os.getenv("PGDATABASE", "socialdb")

MONGO_HOST = os.getenv("MONGO_HOST", "localhost")
MONGO_PORT = int(os.getenv("MONGO_PORT", "27017"))
MONGO_USER = os.getenv("MONGO_USER", "app")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD", "app_pw")

NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "app_pw")

PGHOST, PGPORT, PGUSER, PGDATABASE, MONGO_HOST, MONGO_PORT, NEO4J_URI


## PostgreSQL - create tables

In [ ]:
import psycopg2
from psycopg2.extras import execute_values

conn = psycopg2.connect(
    host=PGHOST, port=PGPORT, user=PGUSER, password=PGPASSWORD, dbname=PGDATABASE
)
conn.autocommit = True
cur = conn.cursor()

# Create tables
cur.execute("""CREATE TABLE IF NOT EXISTS users (
  id SERIAL PRIMARY KEY,
  username TEXT UNIQUE NOT NULL,
  full_name TEXT,
  created_at TIMESTAMPTZ DEFAULT NOW()
);
""")

cur.execute("""CREATE TABLE IF NOT EXISTS posts (
  id SERIAL PRIMARY KEY,
  user_id INTEGER NOT NULL REFERENCES users(id) ON DELETE CASCADE,
  body TEXT NOT NULL,
  created_at TIMESTAMPTZ DEFAULT NOW()
);
""")

print("Tables created.")


### Insert sample users and posts

In [ ]:
# Insert sample data only if empty
cur.execute("SELECT COUNT(*) FROM users;")
count_users = cur.fetchone()[0]
if count_users == 0:
    users = [("alice", "Alice Example"), ("bob", "Bob Sample"), ("carol", "Carol Demo")]
    execute_values(cur, "INSERT INTO users (username, full_name) VALUES %s ON CONFLICT DO NOTHING;", users)

cur.execute("SELECT COUNT(*) FROM posts;")
count_posts = cur.fetchone()[0]
if count_posts == 0:
    posts = [
        (1, "Hello world from Alice"),
        (2, "Bob here, nice to meet you"),
        (1, "Another post by Alice"),
        (3, "Carol checking in"),
    ]
    execute_values(cur, "INSERT INTO posts (user_id, body) VALUES %s;", posts)

print("Sample data present.")


### Query posts with users

In [ ]:
df = pd.read_sql("SELECT p.id, u.username, p.body, p.created_at FROM posts p JOIN users u ON p.user_id=u.id ORDER BY p.id;", conn)
df


## MongoDB - comments collection

In [ ]:
from pymongo import MongoClient

mongo_uri = f"mongodb://{MONGO_USER}:{MONGO_PASSWORD}@{MONGO_HOST}:{MONGO_PORT}/?authSource=admin"
mc = MongoClient(mongo_uri)
mdb = mc["socialdb"]
comments = mdb["comments"]

# Create sample comments if empty
if comments.count_documents({}) == 0:
    comments.insert_many([
        {"post_id": 1, "author": "bob", "text": "Nice first post!", "created_at": pd.Timestamp.utcnow()},
        {"post_id": 1, "author": "carol", "text": "Welcome Alice!", "created_at": pd.Timestamp.utcnow()},
        {"post_id": 2, "author": "alice", "text": "Hi Bob!", "created_at": pd.Timestamp.utcnow()},
    ])
list(comments.find().limit(5))


### Query comments for a given post_id

In [ ]:
pd.DataFrame(list(comments.find({"post_id": 1}, {"_id": 0})))


## Neo4j - relationships

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def run_cypher(query, params=None):
    with driver.session() as session:
        return list(session.run(query, params or {}))

# Constraints and sample data
run_cypher("CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.username IS UNIQUE")
run_cypher("UNWIND $users AS u MERGE (:User {username: u})", {"users": ["alice", "bob", "carol"]})
run_cypher("MATCH (a:User {username:'alice'}), (b:User {username:'bob'}) MERGE (a)-[:FOLLOWS]->(b)")
run_cypher("MATCH (b:User {username:'bob'}), (c:User {username:'carol'}) MERGE (b)-[:FOLLOWS]->(c)")
run_cypher("MATCH (a:User {username:'alice'}), (c:User {username:'carol'}) MERGE (a)-[:FOLLOWS]->(c)")

# Likes - pretend likes map to post ids from Postgres
run_cypher("""MERGE (p1:Post {id: 1})
MERGE (p2:Post {id: 2})
MERGE (p3:Post {id: 3})
MERGE (p4:Post {id: 4})
WITH p1,p2,p3,p4
MATCH (a:User {username:'alice'}), (b:User {username:'bob'}), (c:User {username:'carol'})
MERGE (b)-[:LIKES]->(p1)
MERGE (c)-[:LIKES]->(p1)
MERGE (a)-[:LIKES]->(p2)
MERGE (a)-[:LIKES]->(p4);
""")
print("Graph prepared.")


### Query the follow graph

In [ ]:
rows = run_cypher("MATCH (u:User)-[:FOLLOWS]->(v:User) RETURN u.username AS follower, v.username AS followed ORDER BY follower, followed")
pd.DataFrame([dict(r) for r in rows])


## Cross-database example idea

In [ ]:
# Example: For each Postgres post, attach MongoDB comment count
posts = pd.read_sql("SELECT id, body FROM posts ORDER BY id", conn)
comment_counts = {doc["post_id"]: doc["count"] for doc in comments.aggregate([{"$group": {"_id": "$post_id", "count": {"$sum": 1}}}, {"$project": {"_id": 0, "post_id": "$_id", "count": 1}}])}
posts["comment_count"] = posts["id"].map(comment_counts).fillna(0).astype(int)
posts


### Next steps
- Add pagination queries in Postgres
- Model user profiles in MongoDB
- Model richer relationships in Neo4j
- Consider eventual consistency notes across the three stores